In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.housing_summary
df = pd.DataFrame(list(db.redfin.find().limit(100000)))
# df = pd.DataFrame(list(db.redfin.find()))

In [3]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
'median_sale_ppsf',
'median_new_listing_price',
'pending_sales',
'total_new_listings',
'price_drops',
'homes_delisted',
'inventory',
'total_active_listings',
'median_days_on_market'
]

In [14]:
df_reduced = df[sel_cols]

In [18]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")

df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states

In [23]:
df = df_reduced
df.fillna('', inplace=True)

# df['median_days_on_market'] = df['median_days_on_market'].astype(str)
# df = df[(df.region_name != 'All Redfin Metros') & (df.year == '2021') & (df.median_sale_price != 0) & (df.median_days_on_market < '4000')]
df = df[(df.region_type == 'metro')& (df.year == '2020') & (df.state == 'NY')]
df


,period_begin,period_end,region_name,region_type,total_homes_sold,median_sale_price,median_sale_ppsf,median_new_listing_price,pending_sales,total_new_listings,price_drops,homes_delisted,inventory,total_active_listings,median_days_on_market,year,year_month,state
954,2020-11-02,2020-11-08,"Batavia, NY metro area",metro,11.0,123900.0,86.473132,129900.0,9,9.0,9,,70.0,80.0,34.0,2020,2020-11,NY
1050,2020-12-21,2020-12-27,"Albany, NY metro area",metro,234.0,227000.0,144.698296,242488.5,92,60.0,25,18,1955.0,2090.0,23.0,2020,2020-12,NY
1551,2020-06-22,2020-06-28,"Glens Falls, NY metro area",metro,28.0,233500.0,141.816271,269000.0,34,61.0,19,3,621.0,673.0,98.0,2020,2020-06,NY
1831,2020-06-08,2020-06-14,"Nassau County, NY metro area",metro,404.0,457000.0,314.285714,549000.0,389,1054.0,226,110,9197.0,9937.0,48.5,2020,2020-06,NY
1885,2020-10-12,2020-10-18,"Batavia, NY metro area",metro,16.0,146750.0,85.071655,132450.0,13,12.0,8,1,84.0,99.0,25.5,2020,2020-10,NY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98978,2020-07-13,2020-07-19,"Amsterdam, NY metro area",metro,9.0,170000.0,101.063830,99000.0,10,11.0,5,3,161.0,178.0,65.0,2020,2020-07,NY
99026,2020-06-15,2020-06-21,"Poughkeepsie, NY metro area",metro,89.0,305000.0,155.688623,352500.0,95,296.0,74,20,2207.0,2373.0,54.0,2020,2020-06,NY
99027,2020-03-23,2020-03-29,"Poughkeepsie, NY metro area",metro,70.0,279750.0,156.250000,299999.0,52,67.0,18,24,2183.0,2275.0,85.0,2020,2020-03,NY
99187,2020-09-21,2020-09-27,"Batavia, NY metro area",metro,18.0,140500.0,97.557471,179900.0,10,7.0,7,,80.0,91.0,30.0,2020,2020-09,NY


In [7]:
# df['inventory'] = df['inventory'].astype(str)
# df['inventory'].max()

In [8]:
df.reset_index(inplace = True)
data_dict = df.to_dict("records")

In [9]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)